In [21]:
import json
import numpy as np
import pickle
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

### model training

In [3]:
train_data = []

with open('../data/domain1_train.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        train_data.append(data)

In [22]:
X = [item['text'] for item in train_data]
y = [item['label'] for item in train_data]

In [23]:
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform([" ".join(map(str, text)) for text in X])

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
xgboost_model = xgb.XGBClassifier()

In [31]:
xgboost_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [32]:
y_pred = model.predict(X_test)

In [33]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Acc：{accuracy}')

Acc：0.9041025641025641


In [34]:
with open('../models/xgboost_model.pkl', 'wb') as model_file:
    pickle.dump(xgboost_model, model_file)

In [ ]:
with open('../data/Xgboost_output.csv', 'w') as output_file:
    output_file.write('id,class\n')  # 写入CSV文件的标题行

    # 读取测试集
    with open('../data/test_set.json', 'r') as file:
        for line in file:
            entry = json.loads(line)
            text = entry["text"]

            # 将文本转化为词袋模型特征
            X_test = vectorizer.transform([" ".join(map(str, text))])

            # 使用XGBoost模型进行预测
            prediction = xgboost_model.predict(X_test)

            # 写入CSV文件
            output_file.write(f"{entry['id']},{prediction[0]}\n")